In [4]:
import time
from pynvml import nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
import subprocess

In [5]:
# 初始化NVML
nvmlInit()

def check_gpu_memory(threshold_memory_mb, gpu_index=0):
    """
    检查指定GPU是否有足够的空闲内存。

    参数:
        threshold_memory_mb (int): 程序运行所需的最小空闲内存，单位为MB。
        gpu_index (int): 要检查的GPU编号，默认为0。

    返回:
        bool: 如果可用内存足够返回True，否则返回False。
    """
    handle = nvmlDeviceGetHandleByIndex(gpu_index)
    info = nvmlDeviceGetMemoryInfo(handle)
    free_memory_mb = info.free / 1024**2
    
    print(f"GPU {gpu_index}: Total: {info.total / 1024**2:.2f} MB, "
          f"Used: {info.used / 1024**2:.2f} MB, "
          f"Free: {free_memory_mb:.2f} MB")
    
    return free_memory_mb >= threshold_memory_mb

In [8]:
check_gpu_memory(111)

GPU 0: Total: 81920.00 MB, Used: 64969.38 MB, Free: 16950.62 MB


True

In [6]:
def run_program_if_memory_available(command, threshold_memory_mb, gpu_index=0, wait_time_sec=10):
    """
    当GPU内存足够时运行指定的程序。

    参数:
        command (str): 要运行的程序命令。
        threshold_memory_mb (int): 程序运行所需的最小空闲内存，单位为MB。
        gpu_index (int): 要检查的GPU编号，默认为0。
        wait_time_sec (int): 内存不足时的等待时间，单位为秒，默认为10秒。
    """
    while not check_gpu_memory(threshold_memory_mb, gpu_index):
        print(f"Insufficient GPU memory for command '{command}'. Waiting for {wait_time_sec} seconds...")
        time.sleep(wait_time_sec)
    
    print(f"Running command: {command}")
    subprocess.run(command, shell=True)


In [ ]:
for program in programs_to_run:
    run_program_if_memory_available(
        command=program["command"],
        threshold_memory_mb=program["memory_required_mb"],
        gpu_index=0,  # 可根据需要指定不同的GPU
        wait_time_sec=10
    )